In [1]:
import numpy as np 
import pandas as pd
from functools import reduce

all_subs     = np.array([])
all_pred     = np.array([])
all_dx       = np.array([])
# import os
# print (os.getcwd())

for i in range(5):
    all_subs = np.concatenate((all_subs, np.genfromtxt('../5cv/subid_' + str(i+1) + '.txt', dtype='str')))
    all_dx = np.concatenate([all_dx,  np.loadtxt('../5cv/dx_' + str(i+1) + '.txt')]) 
    all_pred = np.concatenate([all_pred,  np.loadtxt('../5cv/pred_' + str(i+1) + '.txt')]) 
    
    
volums_scores = pd.read_csv('../all_vol_scores.csv')
all_vol = volums_scores[volums_scores.subjectkey.isin(all_subs)]


preds = pd.DataFrame({'subjectkey': all_subs, 'pred': all_pred, 'dx': all_dx}, columns=['subjectkey', 'pred', 'dx'])

dfs = [preds, all_vol]
all_info = reduce(lambda left,right: pd.merge(left,right,on='subjectkey'), dfs)


In [2]:
Y = np.array(all_info['dx'], dtype=np.float64)
X = np.zeros((Y.shape[0],all_info.shape[1]-1-3))

i = 0
for s in all_info.columns[3:all_info.shape[1]-1]:
    X[:,i] = np.array(all_info[s], dtype=np.float64)
    i+=1

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation

predicted = cross_validation.cross_val_predict(LogisticRegression(), X, Y, cv=5)
print (metrics.accuracy_score(Y, predicted))

0.7423870333988212


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators=1500, max_depth=50, random_state=0) #Initialize with whatever parameters you want to

scores_rf = cross_val_score(clf, X, Y, cv=5)
print("RF Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean() * 100.0, scores_rf.std() * 2 * 100.0))

RF Accuracy: 0.71 (+/- 0.03)


In [20]:
print("SVM Accuracy: %0.2f (+/- %0.2f)" % (scores.mean() * 100.0, scores.std() * 2 * 100.0))

SVM Accuracy: 74.09 (+/- 3.09)


In [21]:
print("RF Accuracy: %0.2f (+/- %0.2f)" % (scores_rf.mean() * 100.0, scores_rf.std() * 2 * 100.0))

RF Accuracy: 70.91 (+/- 2.76)
